In [1]:
#First, install all necessary packages:
!apt-get update
!apt-get install -y megahit ncbi-blast+ seqtk emboss mafft

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,201 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran4

In [2]:
#Import compressed (zip) version of files, orelse it takes way too long; upload your highest coverage SRR
from google.colab import files
uploaded = files.upload()

Saving all_contigs.fa.zip to all_contigs.fa.zip


In [3]:
!unzip all_contigs.fa.zip #unzip my all_contigs file

Archive:  all_contigs.fa.zip
  inflating: all_contigs.fa          


In [4]:
!ls -lh #check to see that our unzipped file exists

total 584M
-rw-r--r-- 1 root root 456M Dec  8 15:27 all_contigs.fa
-rw-r--r-- 1 root root 128M Dec 10 17:04 all_contigs.fa.zip
drwxr-xr-x 1 root root 4.0K Nov 20 14:30 sample_data


In [5]:
#Run MEGAHIT assembly for contigs
!mkdir -p /content/megahit_tmp  #temporary directory

!megahit -r all_contigs.fa \
         -o /content/megahit_out \
         --min-contig-len 25 \
         --tmp-dir /content/megahit_tmp

#Zip the output folder & download
!zip -r megahit_out.zip megahit_out
from google.colab import files
files.download('megahit_out.zip')

2025-12-10 17:06:23 - MEGAHIT v1.2.9
2025-12-10 17:06:23 - Using megahit_core with POPCNT and BMI2 support
2025-12-10 17:06:23 - Convert reads to binary library
2025-12-10 17:06:25 - b'INFO  ./src/sequence/io/sequence_lib.cpp:   75 - Lib 0 (/content/all_contigs.fa): se, 2347596 reads, 16590 max length'
2025-12-10 17:06:25 - b'INFO  ./src/utils/utils.h           :  152 - Real: 2.7151\tuser: 2.5671\tsys: 0.3896\tmaxrss: 166540'
2025-12-10 17:06:25 - k-max reset to: 141 
2025-12-10 17:06:25 - Start assembly. Number of CPU threads 2 
2025-12-10 17:06:25 - k list: 21,29,39,59,79,99,119,141 
2025-12-10 17:06:25 - Memory used: 12245258649
2025-12-10 17:06:25 - Extract solid (k+1)-mers for k = 21 
2025-12-10 17:08:19 - Build graph for k = 21 
2025-12-10 17:10:46 - Assemble contigs from SdBG for k = 21
2025-12-10 17:15:23 - Local assembly for k = 21
2025-12-10 17:16:13 - Extract iterative edges from k = 21 to 29 
2025-12-10 17:16:46 - Build graph for k = 29 
2025-12-10 17:17:42 - Assemble conti

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
!ls megahit_out #check to see that our final.contigs.fa has been created

checkpoints.txt  final.contigs.fa      log
done		 intermediate_contigs  options.json


In [7]:
#Create a new fasta file for my rdrp palm sequence
with open("/content/palm.faa", "w") as f:
    f.write(">rdrp_palm\n")
    f.write("MMGVPQEDIQMVEFFWLGHKRLFVKGKFVGTMVNGIPMGDPLTKTCMSLAHAIAHLYAVYKVGAFGRNEGNGDDLTL\n")

In [8]:
#Create a nucleotide database from our MEGAHIT contigs to conduct our analysis
!makeblastdb \
    -in /content/megahit_out/final.contigs.fa \
    -dbtype nucl \
    -out /content/contigs_db



Building a new DB, current time: 12/10/2025 17:44:05
New DB name:   /content/contigs_db
New DB title:  /content/megahit_out/final.contigs.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 116514 sequences in 2.64559 seconds.




In [9]:
!ls -lh /content/contigs_db* #Check contents


-rw-r--r-- 1 root root  20K Dec 10 17:44 /content/contigs_db.ndb
-rw-r--r-- 1 root root  12M Dec 10 17:44 /content/contigs_db.nhr
-rw-r--r-- 1 root root 1.4M Dec 10 17:44 /content/contigs_db.nin
-rw-r--r-- 1 root root 1.4M Dec 10 17:44 /content/contigs_db.not
-rw-r--r-- 1 root root  12M Dec 10 17:44 /content/contigs_db.nsq
-rw-r--r-- 1 root root  16K Dec 10 17:44 /content/contigs_db.ntf
-rw-r--r-- 1 root root 456K Dec 10 17:44 /content/contigs_db.nto


In [10]:
#Finally, tblastn! This can also be done on site, but I am keeping everything in this notebook for ease
!tblastn -query /content/palm.faa \
         -db /content/contigs_db \
         -out /content/tblastn_results.txt \
         -evalue 1e-5 \
         -outfmt 6

In [11]:
!head /content/tblastn_results.txt #Check what contigs are hit!


rdrp_palm	k141_63250	98.701	77	1	0	1	77	412	182	1.97e-53	162
rdrp_palm	k141_70773	100.000	44	0	0	6	49	1	132	4.49e-28	95.5


In [13]:
#Open BLAST results to get matching contig IDs
with open('/content/tblastn_results.txt', 'r') as f:
  hits = f.readlines()
  contig_ids = [hit.split()[1] for hit in hits]

#Extract matching sequences only
with open('/content/megahit_out/final.contigs.fa', 'r') as f:
  contigs = f.read()

#Finally, put only matching contigs into a new file
with open('/content/filtered_contigs.fa', 'w') as f:
    for contig_id in contig_ids:
        start_index = contigs.find(f">{contig_id}")
        if start_index != -1:
            end_index = contigs.find(">", start_index + 1)
            contig = contigs[start_index:end_index] if end_index != -1 else contigs[start_index:]
            f.write(contig + "\n")

In [16]:
#Download my rdrp!
from google.colab import files
files.download('/content/filtered_contigs.fa')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!head -n 10 /content/filtered_contigs.fa #check first 10 bases

head: cannot open '/content/filtered_contigs.fa' for reading: No such file or directory


In [ ]:
#Next, blasgn to see if any other contigs align to our recovered rdrp contig to extend the genome
!blastn -query /content/filtered_contigs.fa -db /content/megahit_out/final.contigs.fa -out /content/blastn_results.txt -evalue 1e-5 -outfmt 6

In [ ]:
#Extract contigs
!grep -A 1 ">k141_24308" /content/megahit_out/final.contigs.fa > /content/contig_k141_24308.fa
!grep -A 1 ">k141_36662" /content/megahit_out/final.contigs.fa > /content/contig_k141_36662.fa
!grep -A 1 ">k141_40107" /content/megahit_out/final.contigs.fa > /content/contig_k141_40107.fa


In [ ]:
#Finally, download. You now have all files needed for rdrp/genome assembly!
from google.colab import files
files.download('/content/contig_k141_24308.fa')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>